In [1]:
import os
os.environ['JAX_PLATFORM_NAME'] = 'cpu'
os.environ['XLA_FLAGS'] = '--xla_force_host_platform_device_count=8'

import time

import jax
import jax.numpy as jnp

from jax.sharding import PartitionSpec as P

In [2]:
print(jax.devices())

[CpuDevice(id=0), CpuDevice(id=1), CpuDevice(id=2), CpuDevice(id=3), CpuDevice(id=4), CpuDevice(id=5), CpuDevice(id=6), CpuDevice(id=7)]


In [3]:
rng = jax.random.PRNGKey(0)
x = jax.random.normal(rng, (8192, 8192))

In [4]:
jax.debug.visualize_array_sharding(x)

                         
                         
                         
                         
                         
          CPU 0          
                         
                         
                         
                         
                         

In [13]:
mesh = jax.make_mesh((8,), ("x"))
y = jax.device_put(x, jax.sharding.NamedSharding(mesh, P("x")))

jax.debug.visualize_array_sharding(y)

          CPU 0          
                         
          CPU 1          
                         
          CPU 2          
                         
          CPU 3          
                         
          CPU 4          
                         
          CPU 5          
                         
          CPU 6          
                         
          CPU 7          
                         

In [14]:
z = jnp.sin(y)
jax.debug.visualize_array_sharding(z)

          CPU 0          
                         
          CPU 1          
                         
          CPU 2          
                         
          CPU 3          
                         
          CPU 4          
                         
          CPU 5          
                         
          CPU 6          
                         
          CPU 7          
                         

In [15]:
%timeit -n 5 -r 5 jnp.sin(x).block_until_ready()

132 ms ± 1.93 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


In [16]:
%timeit -n 5 -r 5 jnp.sin(y).block_until_ready()

74.2 ms ± 3.73 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)
